In [6]:
## imports

import numpy as np
import numpy.ma as ma
import xarray as xr

import sys
sys.path.insert(0,"/home/alberta/Work/git/lupa/vorticity-variance")
import GriddedData

import matplotlib.pyplot as plt

# palette
import matplotlib.cm as mplcm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import glob

In [7]:
## Dataset

dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/EKE/"
dirin="/media/extra/DATA/NATL60/NATL60-I/"


In [8]:
gridfile=dirin+"NATL60_v4.1_cdf_byte_mask.nc"
!ls $gridfile
grid=xr.open_dataset(gridfile)
navlat= grid['nav_lat']
navlon= grid['nav_lon']
tmask='NATL60-CJM165.1d_mask_surf_filt-n80-f0.0125.nc'
mask_filt=np.squeeze(xr.open_dataset(dirin+tmask)['tmask'])


/media/extra/DATA/NATL60/NATL60-I/NATL60_v4.1_cdf_byte_mask.nc


In [9]:
def compute_average_ratio_KE(file_KES,file_KEL):
    ''' from a list of files containing one month of data compute variance and average in 1°x1°x1 month boxes
    '''
    data1 = xr.open_dataset(file_KES)['vomke']
    data2 = xr.open_dataset(file_KEL)['vomke']
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=60)
    data1_mask=np.zeros(data1.shape)
    data2_mask=np.zeros(data2.shape)
    for tt in np.arange(data1_mask.shape[0]):
        truc=np.squeeze(data1[0,:,:].values)        
        truc[np.where(mask_filt!=1)]=np.nan
        data1_mask[tt,:,:]=truc[:,:]   
        truc=np.squeeze(data2[0,:,:].values)        
        truc[np.where(mask_filt!=1)]=np.nan
        data2_mask[tt,:,:]=truc[:,:]   

    data1m=np.nanmean(data1_mask,axis=0)
    data1c = crs.return_ravel(np.asarray(data1m))
    data1cm = np.nanmedian(data1c,axis=-3)

    data2m=np.nanmean(data2_mask,axis=0)
    data2c = crs.return_ravel(np.asarray(data2m))
    data2cm = np.nanmedian(data2c,axis=-3)

    ratio=data1cm/data2cm
    
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)

    return loncrsm,latcrsm,ratio
 

In [17]:
def compute_and_save(month):
    mm=str(month).zfill(2)
    if month < 10:
        year=2013
    else:
        year=2012
    filt_files_month_KES='NATL60-CJM165_y'+str(year)+'m'+str(mm)+'.1m_KES.nc'
    filt_files_month_KEL='NATL60-CJM165_y'+str(year)+'m'+str(mm)+'.1m_KEL.nc'
    tf_files_month_KES=dir+filt_files_month_KES
    tf_files_month_KEL=dir+filt_files_month_KEL
    loncrsm, latcrsm, ratio=compute_average_ratio_KE(tf_files_month_KES,tf_files_month_KEL)
    ratio_da = xr.DataArray(ratio,dims=['xcrsm','ycrsm'],name="ratio_KE_boxcar")
    lat_crsm = xr.DataArray(latcrsm,dims=['xcrsm','ycrsm'],name="latcrsm")
    lon_crsm = xr.DataArray(loncrsm,dims=['xcrsm','ycrsm'],name="loncrsm")
    ratio_da.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='w',engine='scipy')
    lat_crsm.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='a',engine='scipy')
    lon_crsm.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='a',engine='scipy')


In [18]:
for m in np.arange(1,13):
    print m
    compute_and_save(m)

1
2
3
4
5
6
7
8
9
10
11
12
